In [3]:
import numpy as np
from filterpy.kalman import KalmanFilter
from filterpy.kalman import ExtendedKalmanFilter
from filterpy.common import Q_discrete_white_noise



f = KalmanFilter (dim_x=3, dim_z=3)

f.x = np.array([[2,],    # position
                [0.]])   # velocity
f.Q = Q_discrete_white_noise(dim=2, dt=0.1, var=0.13)
